# 使用appnium和路路通移动版获取时刻表  
<b>实际上在开发中appnium已经不在维护且配置较为困难,仅离线使用</b>    
下次会退回重新使用selenium  

## 当前状态  
- 主要由于路路通相较于12306在时刻表层面提供更多的信息  
    - 如停到站时间及始发终到等信息丰富,车站大屏由于网络问题不好使用  
    - 而路路通网页版不提供此类信息(12306不太好作),难以使用web自动化爬取  
    - 这里使用appnium和模拟器来获得数据,但是配置较为复杂  
    - 检票口(映射股道)需要依照车次再次查询如网页版12306或是这里的车次查询

- 目前爬取效果,处理等和前面武汉枢纽相同
    - 基本实现自动化,但是由于车站和车次页面不同,需要手动切换分辨率,也可以开始时直接拉满防止画面溢出,目前暂时不知道更好的方法
    - 目前重点为完善车次爬取部分改为获取停站的上一站和下一站,可以解决进路冲突的问题如 济南东→济南西 和 济南→济南西 均为上行导致无法区分并获取更详细的信息
    - 完善度表格:    <br>

|组件信息|完善程度|当前状态|日后计划|  
|---|---|---|---| 
车站|整体较为成熟完善|可以按照单一地区车站模糊匹配,精确匹配后续筛选|暂无
车次|信息较为完善但是观感不佳|可以获得单一车次给出的大部分信息如停站检票口等.<br>主要为从车次查询-检票口查询获得|考虑压缩为字典样式以更好的存储信息
检票口/股道 | 不完善，需要二次处理|如不需要车站爬取可以直接由单个车站获得足够信息,<br>无需经过查询车次继续处理。若需要检票口信息需要在获取完成车站的车次信息后再查询检票口|完善筛选逻辑
立折车辆 | 纯手动，没有处理|手动查找修改|计划在日后由车次部分下交路表完成|计划在车次查询部分加上交路表部分信息获取
越行车辆 |纯手动，完全没有处理|没有计划|不知道怎么获知越行车信息
  
## 使用的库函数信息和达成的基本效果
<b>主要为了提高复用性功能大体分为了三个部分，数据处理可以单独由excel或csv使用无需使用爬虫</b>  
- 获取整体信息-自动化爬虫部分
    - appnium的APP自动化。<b>实际上在开发中appnium已经不在维护,仅离线使用</b>  
    - 一般的io文本处理部分，输出为csv样式的txt文本文件
- 初步信息处理-数据处理/数据分析
    - 整理出较为规整的数据，主要按照车次来汇总数据便于后续操作
    - 输出符合游戏地图要求的时刻表信息或是进行其他分析  
- 没啥用的可视化
    - 主要是pyecharts，输出一些可交互的图片更为直观

----
## 第一部分，定义使用的库函数和常量信息
### 内容
库函数主要分为三大部分：获取数据--处理数据--<del>分析（可视化）数据</del>。即下面引用部分大体分为了三部分  
而常量部分主要用于处理数据生成符合游戏格式的时刻表，这部分会随着函数的完善而更为精简  
以及最后的模拟器信息用于appnium连接模拟器获取路路通信息  
### 待完善/优化的部分
1. 编组映射待交路表信息获取完成后会按照车型映射如长编和重连，即使用下面marshalling字典
2. 速度等级也会按照车型信息重新映射
3. 建立进场路径映射相关常量来完善分场式车站可能的调度问题，提高调度自动化程度
4. .....

In [7]:
import appium  # app自动化使用，获取信息
from appium import webdriver
from appium.webdriver.webdriver import WebDriver
from appium.webdriver.common.appiumby import AppiumBy
from appium.webdriver.extensions.android.nativekey import AndroidKey
from appium.webdriver.common.touch_action import TouchAction
import time

import pandas

# 数据分析/数据可视化使用
import matplotlib
import pyecharts
import plotly.figure_factory as ff
from pyecharts import options as opts
from pyecharts.charts import Bar, Timeline


trainType = ["普客", "新空调快速", "新空调特快", "新空调直快", "动车", "高速", "城际"]
stationRegion = ["济南", "济南西", "济南东", "大明湖"]


Excelpath = "data.xlsx"  # Excel时刻表输入文件路径，如爬取的时刻表或是是复制来的
TextPath = "train.txt"  # 游戏时刻表文件输出路径
# 速度和编组以及类型映射关系,0为普速1为动车2为高速，后续修改
species = {'新空调普快': ['120', 'LPPPPPP', 0], '新空调快速': ['120', 'LPPPPPP', 0],
           '新空调特快': ['140', 'LPPPPPP', 0], '新空调直快': ['160', 'LPPPPPP', 0],
           '动车': ['200', 'LPPL', 1], '城际': ['200', 'LPPL', 1], '高速': ['300', 'LPPLLPPL', 2]}
species1 = {'K': ['120', 'LPPPPPP', 0], 'T': ['140', 'LPPPPPP', 0], 'Z': ['160', 'LPPPPPP', 0],
            'D': ['200', 'LPPL', 1], 'C': ['200', 'LPPL', 1], 'G': ['300', 'LPPLLPPL', 2]}

# 车站-编号,掉向,用时以及运行车辆种类映射关系
# 图片左(0)右(1)侧线路key值相同则掉向,
# 国铁车辆行走左侧,2为数据为左侧股道编号
# [车站编号,车站所在侧(0为左侧),车辆进场股道,车辆离场行走股道,到达中心车站所用时间]
station = {
    '京济联络线济南方向': ['b', 0, 2, 1, 8], '济郑高速长清方向': ['c', 0, 2, 1, 7],
    '济南西站': ['d', -1, 0, 0, 7], '济南西动车所': ['e', -1, 0, 0, 30],
    '京沪高速德州东方向': ['f', 1, '1', '2', 6, ], '京沪高速泰安站方向': ['a', 0, 2, 1, 7, ],
    '石济客专齐河方向': ['g', 1, 1, 2, 6], '石济客专济南东方向': ['h', 1, 1, 2, 6, ],
}

# 线路和车站关系，主要用于从车站-值获取线路-键
route1 = {
    '济南西动车所': ["济南西"], '京济联络线济南方向': ["济南"], '济郑高速长清方向': ["长清"],
    '京沪高速泰安站方向': ["泰安", "曲阜东", "滕州东", "枣庄", "徐州东", "宿州东", "蚌埠南", "南京南"],
    '京沪高速德州东方向': ["北京南", "天津西", "天津", "沧州西", "德州东"],
    '石济客专齐河方向': ["齐河", "禹城东", "平原东"], '石济客专济南东方向': ["济南东"]}
# 两个合在一起写太难看


# 车型关系--待完善，后续待交路表部分完成后会使用对应映射而不是上面的简单类型映射
marshalling = {'CHR380BL': [], 'CHR380B': [], 'CHR380B重连': []}


ThisStation = '济南西站'
TotalPlat = 18
# track = {}


# 模拟器信息
desired_caps = {
    'platformName': 'Android',  # 被测手机是安卓
    'platformVersion': '9',  # 手机安卓版本
    'deviceName': 'emulator-5554:5555',  # 设备名和映射连接端口，安卓手机可以随意填写

    'appPackage': 'com.lltskb.lltskb',  # 启动APP Package名称
    'appActivity': '.ui.splash.SplashActivity',  # 启动Activity名称

    'unicodeKeyboard': True,  # 使用自带输入法，输入中文时填True
    'resetKeyboard': True,  # 执行完程序恢复原来输入法
    'noReset': True,       # 不要重置App
    'newCommandTimeout': 6000,
    'automationName': 'UiAutomator2'
}


## 第二部分，appnium自动化爬取路路通上时刻表信息
### 内容
这部分主要有三个函数，分别是按照车站获取车次信息，按照车次查询--检票口查询获取该车次所有信息，以及车站大屏查询  
数据均来自路路通，界面信息所见即所得  
1. 车站信息  
    1.1 这部分较为简单，爬取时信息较为规整。始发终到车站信息等界面上每一个格子都对应一个数据，按照正常顺序获取处理即可
    1.2 滑动页面并获取数据，数据按照一行八个获取，以csv样式写入txt
2. 车次信息--检票样式查询  
    2.1 这部分难度就大很多，主要难点在于目前来看必须多次重复点击查询按钮  
    2.2 同样滑动页面并获取数据，数据按照以车站核心一行四个数据，同样以csv样式写入文本txt
3. 车站大屏信息  
    3.1 由于网络问题应该相当长的时间内无法完成，主要目的是为了获取如汉口站，郑州站这类检票口信息不定导致无法离线查询的车站  

### 效果
1. 如果不需要做进路映射和股道映射，单纯通过车站信息爬取已经足够使用，已经有完善的实发终到，到时开时信息
2. 如果需要做进路映射和股道映射且较为复杂，可以用后面车次信息部分来获取这部分信息，但是时长较大

### 实现原理  
1. appnium按流程进入页面并得到文本数据  
2. 死循环获取页面数据，直到页面数据不在变化或者由于数据过少而无法变化直接退出  
3. 使用set来收集数据以优化重复数据  

### 待完善/优化的部分
<del>目标更换 1. 优化车次信息查询逻辑，改善输入格式，进行多轮查询时可以更好的数据结果，为之后按照车次来部署整条线路做准备</del>  
1. 优化车次信息查询效率，尽可能少重复点选
2. 加入点击交路表并获取相关信息的部分，使得立折车次自动合并和正确映射编组成为可能
3. 看看能不能使用车站大屏功能，可以获取停站股道不定的车站检票口停站信息  
4. .......


In [8]:
# 每一个车站的所有停车信息

def StationTrainList(driver: WebDriver, action: TouchAction, station: str):
    time.sleep(8)  # 等待开屏广告
    stationText = driver.find_element(
        AppiumBy.ANDROID_UIAUTOMATOR, 'new UiSelector().text("车站")')

    action.tap(stationText).perform()  # 点击车站选项的位置

    stationChk = driver.find_element(
        AppiumBy.ID, "com.lltskb.lltskb:id/layout_station")  # 车站选择部分
    action.tap(stationChk).perform()  # 进入车站选择
    time.sleep(1)
    stationSelect = driver.find_element(
        AppiumBy.ID, "com.lltskb.lltskb:id/edit_input")  # 车站选择输入框
    stationSelect.send_keys(station)  # 输入

    stationFind = driver.find_elements(
        AppiumBy.ANDROID_UIAUTOMATOR, 'new UiSelector().text("{0}")'.format(station))
    action.tap(stationFind[1]).perform()  # 输入车站名并确认，返回之前的页面

    searchbtn = driver.find_element(
        AppiumBy.ID, "com.lltskb.lltskb:id/querycz_btn")  # 搜索按钮
    action.tap(searchbtn).perform()  # 点选进行查找
    time.sleep(2)
    # 当前页面可视的车次部分
    pt = driver.find_element(AppiumBy.CLASS_NAME, 'android.widget.ListView')
    x1 = pt.size['width'] * 0.5
    y1 = pt.size['height']  # 获取页面宽高,方便后续滑动截取车次信息

    f = open(file="{0}Info.txt".format(station), mode="w", encoding="utf-8")
    # 设置为utf8，防止如复兴号，静音动车等符号导致报错
    settext = set()  # 使用set尽可能减少重复值

    while True:  # 滑动截取文字
        # 所有有文本的元素的集合，均为当前可见部分
        pageTrainList = driver.find_elements(
            AppiumBy.CLASS_NAME, 'android.widget.TextView')
        beforeSwipe = driver.page_source

        text = ""
        for i in range(0, len(pageTrainList)-9):  # 排除表头信息，仅保留车次部分
            text = text+pageTrainList[i+9].text+","  # 获取整体信息
            if (i+9) % 8 == 0:

                settext.add(text+"\n")
                text = ""

        try:
            # 这里模拟器的分辨率是1920*1080的手机屏幕，不一样的需要改参数
            # 如果内容少于一屏幕会直接报错，故使用try
            action.long_press(
                x=540, y=1800, duration=200).move_to(x=540, y=100).release()
            action.perform()  # 在中间滑动，选取新的车次信息
        # 如果滑动前后的元素相同，则表示已经到底了
        except Exception:  # 少于一屏幕就表示已经完成了
            break
        else:  # 前后的页面元素一样
            if driver.page_source == beforeSwipe:
                break
    for it in settext:
        f.write(it)
    f.close()

    return

# 一组一列车的所有停站信息


def TrainStationList(driver: WebDriver, action: TouchAction, trainli: list[str], fn="appniumTrainInfo.txt"):
    time.sleep(8)  # 等待开屏广告
    stationText = driver.find_element(
        AppiumBy.ANDROID_UIAUTOMATOR, 'new UiSelector().text("车次")')
    action.tap(stationText).perform()  # 点击车次选项的位置

    stationchk = driver.find_element(
        AppiumBy.ID, "com.lltskb.lltskb:id/chk_jpk")
    action.tap(stationchk).perform()  # 点选查找检票口选项
    # 设置为utf8，防止如复兴号，静音动车等符号导致报错
    f = open(file=fn, mode="w", encoding="utf-8")
    for train in trainli:
        settext = set()  # 使用set尽可能减少重复值
        btnClicked = []
        trainSelect = driver.find_element(
            AppiumBy.ID, "com.lltskb.lltskb:id/edit_train")
        trainSelect.clear()  # 清空内容防止重输
        trainSelect.send_keys(train)  # 输入查找车次

        searchbtn = driver.find_element(
            AppiumBy.ID, "com.lltskb.lltskb:id/querycc_btn")  # 搜索按钮
        action.tap(searchbtn).perform()  # 点选进行查找

        text = ""
        while True:  # 进入页面
            searchbtns = driver.find_elements(
                AppiumBy.ID, "com.lltskb.lltskb:id/btn_query")  # 搜索按钮
            beforeSwipe = driver.page_source

            for btn in searchbtns:  # 当前页面上所有“查询”按钮，目前除了全部重复点一遍没有好办法
                # if btn.id in btnClicked:
                #     continue
                action.tap(btn).perform()  # 点选所有按钮进行查找
                btnClicked.append(btn.id)  # 只能用列表等来标记

            pageStationList = driver.find_elements(
                AppiumBy.CLASS_NAME, 'android.widget.TextView')
           # 获取页面所有信息
            for i in range(1, len(pageStationList)):
                # 停站，到时，开时，检票口共四个停站信息
                text = text+pageStationList[i].text+","
                if i % 4 == 0:  # 每一个车站
                    text = train+","+text+"\n"
                    settext.add(text)
                    text = ""
            try:
                # 这里模拟器的分辨率是1920*1080的手机屏幕
                # 如果内容少于一屏幕会直接报错，故使用try
                action.long_press(
                    x=540, y=1700, duration=150).move_to(x=540, y=200).release()
                action.perform()  # 在中间滑动，选取新的车次信息
                time.sleep(0.5)
            # 如果滑动前后的元素相同，则表示已经到底了
            except Exception:  # 少于一屏幕就表示已经完成了
                print(Exception.args)
                break
            else:  # 前后的页面元素一样
                if driver.page_source == beforeSwipe:
                    break

        # 按下返回按键，进行下一次查找
        retbtn = driver.find_element(
            AppiumBy.ID, "com.lltskb.lltskb:id/btn_back")
        action.tap(retbtn).perform()
        # 替换两个汉字，方便后续操作
        print(settext)
        for it in settext:
            it = it.replace("到", "").replace("开", "")
            f.write(it)
        f.flush()  # 直接刷新缓冲区写入

    f.close()

    return


# 未完成，车站大屏查询


def StationScreenList(driver: WebDriver, action: TouchAction, train: str):
    stationText = driver.find_element(
        AppiumBy.ANDROID_UIAUTOMATOR, 'new UiSelector().text("车站")')

    action.tap(stationText).perform()  # 点击车站选项的位置

    stationChk = driver.find_element(
        AppiumBy.ID, "com.lltskb.lltskb:id/btn_big_screen")  # 车站大屏选择
    action.tap(stationChk).perform()  # 车站大屏内容部分

    return


<b> 爬取部分无需多次运行 </b>

In [9]:


driver = webdriver.Remote('http://localhost:4723/wd/hub', desired_caps)
# 设置缺省等待时间
driver.implicitly_wait(10)
action = TouchAction(driver)  # 点选操作的对象

targetSt = "济南西"
# 济南Info是之前爬取的济南地区车站信息
stationFrame = pandas.read_csv(filepath_or_buffer="济南Info1.txt",
                               sep=",", encoding="utf-8")
stationFrame.columns = ["车次", "drop1", "停站",
                        "到时", "开时", "始发站", "终到站", "列车类型", "drop2"]
# 删去重复列和空列
stationFrame.drop_duplicates(inplace=True)
stationFrame.drop(columns=["drop1", "drop2"], inplace=True)

# 获取所有济南西站的车次信息
jnzlist = stationFrame[stationFrame["停站"] == targetSt]["车次"].to_list()
TrainStationList(driver, action, jnzlist, "jinanxitest2.txt")

time.sleep(3)
# driver.close()
driver.quit()


{'G376,镇江南,08:56到,08:58开,无检票口信息,\n', 'G376,衡水北,13:25到,13:28开,无检票口信息,\n', 'G376,南京南,09:16到,09:20开,检票口A6,\n', 'G376,昆山南,07:49到,07:55开,检票口2,\n', 'G376,济南西,12:04到,12:07开,检票口17B,\n', 'G376,德州东,12:48到,12:51开,检票口2,\n', 'G376,苏州北,08:06到,08:20开,检票口1A,\n', 'G376,平原东,12:35到,12:37开,无检票口信息,\n', 'G376,辛集南,13:44到,13:46开,无检票口信息,\n', 'G376,泰安,11:42到,11:47开,检票口检票口1 检票口2,\n', 'G376,景州,13:07到,13:09开,无检票口信息,\n', 'G376,滁州,09:38到,09:40开,无检票口信息,\n', 'G376,上海虹桥,--:--到,07:32开,检票口7B,\n', 'G376,徐州东,10:47到,10:49开,检票口东广场进站7A7B检票,\n', 'G376,石家庄,14:14到,--:--开,无检票口信息,\n', 'G376,宿州东,10:26到,10:28开,无检票口信息,\n'}
{'G301,德州东,11:39到,11:41开,检票口2,\n', 'G301,福州,19:49到,19:52开,检票口8检票口,\n', 'G301,无检票口信息,上饶,18:10到,18:13开,\n', 'G301,合肥北城,15:02到,15:04开,无检票口信息,\n', 'G301,延平,19:11到,19:13开,无检票口信息,\n', 'G301,曲阜东,12:42到,12:57开,无检票口信息,\n', 'G301,上饶,18:10到,18:13开,检票口一层5A号检票口,\n', 'G301,济南西,12:05到,12:10开,检票口5B,\n', 'G301,武夷山北,18:35到,18:37开,无检票口信息,\n', 'G301,无检票口信息,德兴,17:50到,17:52开,\n', 'G301,检票口1层检票,婺源,17:37到,17:39开,\n', 'G301,无检票口信息,绩溪北,16:5

StaleElementReferenceException: Message: Cached elements 'By.id: com.lltskb.lltskb:id/btn_query' do not exist in DOM anymore
Stacktrace:
io.appium.uiautomator2.common.exceptions.StaleElementReferenceException: Cached elements 'By.id: com.lltskb.lltskb:id/btn_query' do not exist in DOM anymore
	at io.appium.uiautomator2.model.ElementsCache.restore(ElementsCache.java:78)
	at io.appium.uiautomator2.model.ElementsCache.get(ElementsCache.java:153)
	at io.appium.uiautomator2.handler.GetSize.safeHandle(GetSize.java:42)
	at io.appium.uiautomator2.handler.request.SafeRequestHandler.handle(SafeRequestHandler.java:59)
	at io.appium.uiautomator2.server.AppiumServlet.handleRequest(AppiumServlet.java:266)
	at io.appium.uiautomator2.server.AppiumServlet.handleHttpRequest(AppiumServlet.java:260)
	at io.appium.uiautomator2.http.ServerHandler.channelRead(ServerHandler.java:68)
	at io.netty.channel.AbstractChannelHandlerContext.invokeChannelRead(AbstractChannelHandlerContext.java:366)
	at io.netty.channe

## 第三部分，处理获取到的数据并生成游戏样式的时刻表
这部分最为重要，按照游戏中时刻表格式分为若干部分  
车辆信息部分，如车次编号，速度等级等信息  
停站信息部分，有四部分按顺序分别为 停站编号，停站股道，停站时刻，停站时间。这四部分会单独拆开分别生成  
例： G7 COMMUTER 300 LPPLLPPL X1 : f#1#11:17:00#0 d#6#11:23:00#3 a#1#11:33:00#0  

内容|完成度|实现原理|近期改良计划|远期优化计划
---|---|---|---|---
车辆信息|较为完善|目前为仅按照车次编号和类型生成|近期暂无|完善交路表爬取以修改速度级和编组信息立折车次信息完善
停站信息-停站编号|完善|依照游戏自身设置直接映射|暂无|暂无
停站信息-停站股道|较为完善|selenium按照web自动化12306查询获取 <br>appnium部分按照车次信息查询--检票口获得|完善车次查询部分以获得<br>停站检票口信息|使用车站大屏来获得无法离线查询的车站信息
停站信息-停站时刻|完善|按照到站时间和出发时间推算|近期暂无|加上线路所映射行车进路部分
停站信息-停站时间|完善|按照到站时间和出发时间计算|近期暂无|立折车次停站时间等按照交路表推算
整体上|较为完善|使用apply(lambda) 来进列映射整处理而不是按照行单个处理|暂无  

### 第三部分第〇小部分--处理用函数定义
#### 内容  
目前按照上面输入为上面自动化爬取获取到csv样式的txt文件，输出为按照车站为单位的数据  
1. 首先去除空行和重复值等基本操作进行数据提纯   
2. 在统一处理如得到部分信息如停站时间和始发终到的处理，  
3. 可以获得中心车站的停站时间信息等  
4. 同时预先定义需要使用的处理函数  

#### 实现原理  
1. 整体上以处理dataframe为主，文件→dataframe→文本
2. 大量使用.apply相应处理函数来处理列，尽可能少使用匿名函数来增加可读性
3. 由文件读出的大表来初步处理groupby 再对想到较小的表来处理
4.


#### 待完善/优化的部分
近期暂无计划，远期计划加入进路映射以使用线路所来优化自动化  

In [10]:
# 数据处理用
import pandas
import numpy
import time
import datetime
import random
import itertools

def dealTime(datafr: pandas.DataFrame) -> pandas.DataFrame:
    # 对于始发车和终到车进行时间处理，默认相同
    for idx, row in datafr.iterrows():
        if row["到时"] == "--:--":
            row["到时"] = row["开时"]
        elif row["开时"] == "--:--":
            row["开时"] = row["到时"]
    # 计算停站时间
    datafr["到时"] = pandas.to_datetime(datafr["到时"])
    datafr["开时"] = pandas.to_datetime(datafr["开时"])
    # 实发终到设置停站时间
    # pd["停站时间"].replace(0,10,inplace=True)

    return datafr


def stopStTime(stopTime: pandas.Timedelta) -> int:
    a = int(stopTime.seconds/60)  # 处理为数值类型的分钟时间
    # 始发或是终到的时间是随机的
    return a if a > 0 else random.randint(10, 30)


def ModtimeStr(switchTime: pandas.Timestamp) -> str:
    # 处理为仅有时分秒样式的字符串格式的时间
    return switchTime.strftime("%H:%M:%S")


def speMarType(traincode: str) -> str:  # 目前获取编组信息的方式
    til = species1[traincode[0]]  # 处理获取车辆信息字符串部分
    return "{cod} COMMUTER {speed} {mar} X1".format(cod=traincode, speed=til[0], mar=til[1])


def checkin(entrance: str) -> int:
    # 终到车
    res = 0
    if entrance == "无检票口信息":
        res = 0
    else:  # 格式类似检票口15A，替换掉其他字样
        res = int(entrance.strip("检票口AB"))
    return res


def prevnextST(stopSt: list[str], stIdx: int) -> list[str]:
    # 返回结果，获取目标车站的前后车站
    res = ["", ""]
    res1 = ["", ""]
    if stIdx == 0:  # 始发车视为始发车站和下一站
        res = [stopSt[0], stopSt[1]]
    elif stIdx == len(stopSt)-1:  # 终到车视为前一站和终到站
        res = [stopSt[len(stopSt)-2], stopSt[len(stopSt)-1]]
    else:  # 中间站
        res = [stopSt[stIdx-1], stopSt[stIdx+1]]
    # 判断并修改进路
    for k, v in route1.items():
        # 把车站名映射为线路
        if res[0] in v:
            res1[0] = k
        elif res[1] in v:
            res1[1] = k
    return res1


def arrLeaTime(t1: pandas.Timestamp, st: list[str], mark: int) -> str:
    # 选取是上一站还是下一站
    tarst = st[mark]
    useTime = station[tarst][4]
    if mark == 0:  # 进场减时间
        res = t1-datetime.timedelta(minutes=useTime)
    else:  # 离场加时间
        res = t1+datetime.timedelta(minutes=useTime)
    # 返回时分秒格式的字符串
    return res.strftime("%H:%M:%S")


### 第三部分第一小部分--爬取信息整体初步处理 
#### 内容
1. 初步整理上面按照<b>车次检票口查询</b>到的数据
2. 按照车次-进站时间整理后进行分组，按照车次聚合为数据框
3. 获取上下站的信息等，完成初步处理

#### 实现原理
缺省值填充等数据清洗操作  
重分组来重新构造数据特征  
#### 待完善/优化的部分
暂无

In [11]:
# 手动建立映射关系,别的方法不会


def processTrains(fn: str, targetSt: str) -> pandas.DataFrame:
    # 处理产生初步所需信息的数据框
    trainsFrame = pandas.read_csv(filepath_or_buffer=fn,
                                  sep=",", encoding="utf8", header=None)
    trainsFrame.columns = ["车次",  "停站", "到时", "开时", "检票口", "drop1"]
    trainsFrame.drop(columns=["drop1"], inplace=True)

    trainsFrame = dealTime(trainsFrame)  # 初步处理时间
    trainsFrame.sort_values(by=["车次", "到时"], inplace=True)  # 归类排序
    # 整理为分钟样式的停站时间
    #trainsFrame["停站时间"] = trainsFrame["停站时间"].apply(lambda x: stopStTime(x))
    trainsFrame.reset_index(drop=True, inplace=True)
    # 按照车次分组为列表，同时车次不作为索引以便于操作
    tfgp = trainsFrame.groupby(by=["车次"], as_index=False).agg(list)
    # 获取目标停站的索引值，设置为一个辅助列
    tfgp["auxIdx"] = tfgp['停站'].apply(lambda x: x.index(targetSt))

    # 获取前后停站以得到更好的交路映射
    tfgp["目标前后站"] = tfgp.apply(
        lambda x: prevnextST(x["停站"], x["auxIdx"]), axis=1)
    # 获取其他信息
    tfgp["目标站到时"] = tfgp.apply(lambda x: x["到时"][x["auxIdx"]], axis=1)
    tfgp["目标站开时"] = tfgp.apply(lambda x: x["开时"][x["auxIdx"]], axis=1)
    tfgp["目标站检票口"] = tfgp.apply(lambda x: x["检票口"][x["auxIdx"]], axis=1)

    # 提取需要的信息为新的dataframe
    trainInfoNeed = tfgp[["车次", "目标站到时", "目标站开时", "目标站检票口", "目标前后站"]]
    trainInfoNeed.columns = ["车次", "到时", "开时", "检票口", "前后站"]
    # print(trainInfoNeed.head())

    return trainInfoNeed


###  第三部分第二小部分 -- 处理生成最终dataframe样式的数据 

#### 内容
按照上面的函数处理数据，最终处理生成有进(离)场时间/股道 等信息的dataframe,之后生成字符串导出    
这里分为两部分，车辆信息的trainframe和路径时刻表的stationframe,    
 'D8176 COMMUTER 200 LPPL X1 : b#2#18:08:00#0 d#0#18:16:00#14 e#0#18:36:00#0 ',  
<b>如果不做股道映射直接在等号后面写0即可，或是使用如随机数做映射等操作</b>  

#### 实现原理
> 1. generateTrainInfo 即为文字版时刻表分号前的内容，生成车次信息部分  
>> 1.1. 考虑到内容只需做车次编号，速度等级和编组信息则按照车次首位ktz生成    
>> 2.1. 编组信息尚未准备开始   
> 2. generateArriveLeave 即为文字版时刻表分号后的内容，生成进场和离场信息部分  
>> 2.1. 基于停站的上一站完善进路的刻画
>> 2.2. 本身进场和离场处理基本类似，这里使用一个mark来进行区分
> 3. generateMainSt 即为区域核心车站生成部分  
>> 3.1. 绝大多数信息已经在前面处理完成，这里主要是处理格式问题  
>> 3.2. 车站名称的填充放在最后，防止由于数据全空导致填充失败  

#### 改进计划  
1. 对于分场式车站采用进场线路--对应线路所来映射，加入线路所映射部分


In [12]:


def generateRoute(dataser: pandas.Series):
    # 进行中，对于分场式车站采用线路所映射进路
    s1 = "黄河南线路所"
    s2 = "大漠刘线路所"
    return None
#

def generateArriveLeave(datafr: pandas.DataFrame, mark: int) -> pandas.DataFrame:
    # 生成进场/离场信息
    # mark=0为进场，mark=1为离场
    ALStDF = pandas.DataFrame(columns=['车站名称', '股道', '到达时间', '停站时间'])  # 进场信息
    # 按照信息来映射进场/离场线路
    ALStDF['车站名称'] = datafr["前后站"].apply(lambda x: x[mark])
    # 按照信息来映射进场/离场股道
    ALStDF['股道'] = ALStDF['车站名称'].apply(lambda x: station[x][2+mark])
    # ALStDF['股道'] = 0 #不做股道映射则直接填0即可
    # 时间作差
    ALStDF['到达时间'] = datafr.apply(
        lambda x: arrLeaTime(x["到时"], x["前后站"], mark), axis=1)
    # 进程默认经过不停车
    ALStDF['停站时间'] = 0

    return ALStDF


def generateMainSt(datafr: pandas.DataFrame):
    # 生成中心停站信息
    MainStDF = pandas.DataFrame(columns=['车站名称', '股道', '到达时间', '停站时间'])  # 进场信息

    # 计算停站时长，得到整形类型的时长，非浮点数没有.0便于写入文件
    MainStDF["停站时间"] = (datafr["开时"]-datafr["到时"]
                        ).apply(lambda x: stopStTime(x))
    # 计算并规整进场离场和停站时间，将三个到时改为游戏时分秒格式 hh:mm:ss
    MainStDF["到达时间"] = datafr["到时"].apply(lambda x: ModtimeStr(x))
    # 按照相应格式处理车站检票口为对应形式
    MainStDF["股道"] = datafr["检票口"].apply(lambda x: checkin(x))
    # 填充车站为中心车站
    MainStDF["车站名称"].fillna(value="济南西", inplace=True)
    return MainStDF


def generateTrainInfo(dataser: pandas.Series) -> pandas.Series:
    # 生成车次信息
    res = dataser.apply(lambda x: speMarType(x))
    print(res.head())
    return res


## 第四部分 -- 生成对应样式的字符串并写入文件

### 内容  
1. 整合之前的数据来生成符合样式的dataframe数据合集  
2. 生成csv长字符串并按行分割为一个个的端数据  
3. 最终连接处理生成结果，不替换代号便于后续检查    

### 实现原理
整合数据和字符串处理

### 改进计划  
暂无

In [13]:

def generateStr(fn: str) -> list:
    # 生成游戏样式的字符串
    f = open(file=fn, encoding="utf8", mode="w")
    res = []
    for i in range(0, len(arriveStDF)):
        # format为对应顺序格式
        tf = "{train} : {arrive} {stop} {leave}".format(
            train=trainstr[i], arrive=arrivestr[i], stop=stopstr[i], leave=leavestr[i])
        print(tf)
        res.append(tf)  # 将结果写入文件
        f.write(tf+"\n")

    f.close()
    return res

# 实际运行部分
targetSt = "济南西"
at = processTrains("jinanxitest1.txt", targetSt)
# 生成dataframe格式的数据
arriveStDF = generateArriveLeave(at, mark=0)
leaveStDF = generateArriveLeave(at, mark=1)
stopStDF = generateMainSt(at)
trainDF = generateTrainInfo(at["车次"])

# 按照四部分生成需求的字符格式，使用to_csv函数来生成
arrivestr = arriveStDF.to_csv(
    sep="#", header=False, index=False).split(sep="\r\n")
leavestr = leaveStDF.to_csv(sep="#", header=False,
                            index=False).split(sep="\r\n")
stopstr = stopStDF.to_csv(sep="#", header=False, index=False).split(sep="\r\n")
trainstr = trainDF.to_csv(header=False, index=False).split(sep="\r\n")

generateStr("appnium{0}测试.txt".format(targetSt))


ParserError: Error tokenizing data. C error: Expected 6 fields in line 57, saw 7



-------  

## 第五部分，可视化用于分析分布如高峰期和主要类型

In [ ]:

# 对于该车站群组（地区）时间段内的所有车次信息


def getStationProid(datafr: pandas.DataFrame, at: int) -> dict:
    # 预先分类，按照小时
    datagruop = datafr.groupby(["到时-小时", "停站", "列车类型"])
    # print(datagruop.groups)
    regionInfo = {}

    # 对于每个车站的每种类型的列车
    for sr in stationRegion:
        # 好像只能分开写两层了
        regionInfo[sr+"trainCode"] = []
        regionInfo[sr+"trainCount"] = []
        for tt in trainType:
            try:  # 查找符合条件的群组
                i = datagruop.get_group((at, sr, tt))

            except KeyError:  # 索引找不到就是0
                regionInfo[sr+"trainCount"].append(0)
                # regionInfo["trainCode"].append(None)
            else:  # 如果没问题，添加回去长度

                regionInfo[sr+"trainCode"].append(i["车次"].tolist())
                regionInfo[sr+"trainCount"].append(len(i))
        regionInfo[sr+"trainCode"] = sum(regionInfo[sr+"trainCode"], [])
    # 解包嵌套的列表
    # print(regionInfo)

    return regionInfo


def getTrainProid(datafr: pandas.DataFrame, at: int) -> dict:
    # 预先分类，数据分析用
    datagruop = datafr.groupby(["到时-小时", "停站", "列车类型"])
    # print(datagruop.groups)
    regionInfo = {}

    # 对于每个车站的每种类型的列车
    for tt in trainType:
        # 好像只能分开写两层了
        regionInfo[tt+"trainCode"] = []
        regionInfo[tt+"stationName"] = []
        regionInfo[tt+"stationCount"] = []
        for st in stationRegion:
            try:  # 查找符合条件的群组
                i = datagruop.get_group((at, st, tt))

            except KeyError:  # 索引找不到就是0
                regionInfo[tt+"stationCount"].append(0)
                # regionInfo["trainCode"].append(None)
            else:  # 如果没问题，添加回去长度
                regionInfo[tt+"trainCode"].append(i["车次"].tolist())
                regionInfo[tt+"stationName"].append(i["停站"].tolist())
                regionInfo[tt+"stationCount"].append(len(i))

        regionInfo[tt+"stationName"] = sum(regionInfo[tt+"stationName"], [])
    # 解包嵌套的列表
    # print(regionInfo)

    return regionInfo
